In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Columbia EECS E6893 Big Data Analytics


from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import datetime
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
from kafka import KafkaProducer
import datetime

def get_reset_seconds():
    date_second = (datetime.datetime.now() + datetime.timedelta(seconds=30)).strftime("%Y-%m-%d") + ' 00:00:00'

    reset_ts = int(datetime.datetime.strptime(date_second, "%Y-%m-%d %H:%M:%S").timestamp())

    return reset_ts - int(datetime.datetime.now().timestamp())

TOPIC = "tweetss"


# parameter
IP = 'localhost'    # ip port
PORT = 9001     # port

STREAMTIME = 10          # time that the streaming process runs

WORD = ['percent', 'price', 'market', 'sale', 'rise',
        'rate', 'share', 'stock', 'sell', 'increase']     #the words you should filter and do word count



def stock_filter(line):

    def isEnglish(s):
        try:
            s.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True
    
    def containStock(s):
        for word in WORD:
            if word in s:
                return True
        return False
    
    

    line = line.map(lambda x: x.lower()).filter(lambda x: isEnglish(x)).filter(lambda x: containStock(x))
#     line = line.map(lambda x: (x, (datetime.datetime.now().timestamp() - start_time.value) // STREAMTIME))
    return line
    

    
def kafka_sink(lines):
    producer = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER, 
                             api_version=(0,11,5))
    data = lines.collect()
    for msg in data:
        print(msg)
        producer.send("tweets", bytes(msg, encoding="utf8"))
    
    return lines
    

def handler(message):
    records = message.collect()
    if len(records) <= 0: return
    idx = int(records[0][1])
    with open("data{0}.txt".format(idx), "a") as f:
        for record in records:
            idx = record[1]
            msg = record[0]
            f.write(msg + "\n")


KAFKA_BOOTSTRAP_SERVER = "localhost:9092"


if __name__ == '__main__':

    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    sc = SparkContext(conf=conf)
#     sc.setLogLevel("ERROR")
    sc.setLogLevel("OFF")
    start_time = sc.broadcast(datetime.datetime.now().timestamp())
    


    sql_context = SQLContext(sc)
    ssc = StreamingContext(sc, 10)
    # setting a checkpoint to allow RDD recovery
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # read data from port 9001
    dataStream = ssc.socketTextStream(IP, PORT)


    print("---------checkpoint{0}-----------".format(0))



    res = stock_filter(dataStream)
#     res.pprint()
    
    res.foreachRDD(lambda x: kafka_sink(x))
    ssc.start()

    ssc.awaitTerminationOrTimeout(get_reset_seconds()) 
    time.sleep(STREAMTIME)

    ssc.stop(stopSparkContext=False, stopGraceFully=True)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


---------checkpoint0-----------


21/12/22 20:26:21 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:21 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204781400 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:21 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:21 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204781600 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:22 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:22 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204781800 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:22 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:22 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204782000 replicated to only 0

21/12/22 20:26:28 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:28 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204788400 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:29 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:29 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204788800 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:29 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:29 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204789000 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:26:29 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:26:29 WARN org.apache.spark.storage.BlockManager: Block input-0-1640204789200 replicated to only 0

https://t.co/alwrd7aynlrt @chrisexcel102: unpopular fact.
cassper nyovest is using slik talk to market don billiato.
how to share your wish:
2) sell the rip in a downtrend
sell
i was selling
sell myself
there's tmi so just concentrate on the rsi from the previous tweet:
inclusions and price reveal
cassper nyovest is using slik talk to market don billiato.
this is no longer about the match.rt @duffman_d: new #monsterenergy #kenblock43 livery for the hoonigan ford escort cosworth v.2 shared on forza horizon 5.
price: $45,000
price: $583.94 
with special price
how to share your wish:
price 70 baht 


...

-sale of rotowire
2nd place increase: 789
price: 0.025 eth 
whitelist registration for private sale is live now!
save success rate | 85% (1st)
how to share your wish:
to cheer for crypto market,
* 012-9442027 *. please help share this around!
energy prices
floor price 0.002
cassper nyovest is using slik talk to market don billiato.
this is no longer about the match.rt @gamingbyframe: share of the year #psblog #psshare 
how to share your wish:
hospitalizations up 15% (lowest rate of growth in weeks!)
i'm so excited to share robert from whispers in the woods with you!
https://t.co/qolxrupvejrt @blue_footy: harvey vale looks ready for men's football. he has the confidence, the ability and work-rate required.rt @portal_finance: portal defi whitelist allocation event
please fill the airdrop &amp; private sale forms.


21/12/22 20:34:31 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:31 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205271000 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:31 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:31 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205271200 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:31 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:31 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205271400 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:31 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:31 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205271600 replicated to only 0

21/12/22 20:34:38 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:38 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205278000 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:38 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:38 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205278200 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:38 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:38 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205278400 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:38 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:38 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205278600 replicated to only 0

everyone please share 
very excited to see this release on the market,
rate per night:
day 2 of mythmas and we have more pre-sale spots to giveaway!
share christmas-themed photos or videos of #binance using the hashtag #binancechristmas.
how to share your wish:
3 lucky winners will share $15 usdt rewards
please share. https://t.co/mumrcohhobrt @thepoetjean: goal: $1000 for emergency household bills
after one sale from my music nft collection i will be giving away this lil wayne derivative 
check thread! comment or dm if interested! https://t.co/y10r3naunort @watcherguru: #shib prediction: price awaits lift-off as bulls test 50-sma
how to share your wish:
how to share your wish:


21/12/22 20:34:41 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:41 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205280800 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:41 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:41 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205281000 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:41 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:41 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205281200 replicated to only 0 peer(s) instead of 1 peers
21/12/22 20:34:41 WARN org.apache.spark.storage.RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
21/12/22 20:34:41 WARN org.apache.spark.storage.BlockManager: Block input-0-1640205281400 replicated to only 0